In [1]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from operator import itemgetter
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

In [3]:
branch = RunnableBranch(
                (lambda x: x == 'hello', lambda x: x),
                (lambda x: isinstance(x, str), lambda x: x.upper()),
                (lambda x: "This is the default case, in case no above lambda functions match."),
            )

print(branch.invoke("hello")) # "hello"
print(branch.invoke(None)) # "This is the default case"

hello
This is the default case, in case no above lambda functions match.


In [4]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI(model = "gpt-4o-mini")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI(model = "gpt-4o-mini")
    | StrOutputParser()
)

arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI(model = "gpt-4o-mini")
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI(model = "gpt-4o-mini")
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [5]:
chain.invoke({"input": "scrum"})

"In evaluating the adoption of Scrum as a project management framework, it is crucial to consider both its advantages and disadvantages comprehensively. The positive aspects of Scrum—including enhanced collaboration, incremental delivery, flexibility, transparency, and a focus on user needs—demonstrate its potential to significantly improve project outcomes in dynamic environments. However, acknowledging the inherent challenges is equally important for successful implementation.\n\n**Addressing the Critique of Scrum** \n\n1. **Training and Initial Learning Curve**: To mitigate the steep learning curve associated with Scrum, organizations should invest in comprehensive training programs and workshops. Providing access to resources and experienced Scrum trainers can help teams transition smoothly and quickly become proficient in the framework.\n\n2. **Overemphasis on Meetings**: While regular meetings are integral to Scrum, organizations should strive to ensure they are purposeful and ef

In [7]:
joke_chain = (
    ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
    | {"joke": RunnablePassthrough(), "topic": RunnablePassthrough()}
)

explain_joke = (
    ChatPromptTemplate.from_template("Explain the joke: {joke}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

benefits_of_joke = (
    ChatPromptTemplate.from_template("List the benefits of this joke: {joke}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [   
            ("system", "You are responsible for generating a small analysis of a joke. The topic will be: {topic}"),
            ("ai", "{joke}. The benefits of this joke are: {benefits}"),
            ("human", "The explanation of the joke is: {explanation}"),
            ("human", "Generate a small analysis of the joke. Analysis: "),
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

final_chain = (
    {"topic": RunnablePassthrough()}
    | joke_chain
    | {
        "explanation": explain_joke,
        "benefits": benefits_of_joke,
        "joke": itemgetter("joke"),
        "topic": itemgetter("topic"),
    }
    | final_responder
)

final_chain.invoke({"topic": "bears"})

'**Analysis**: The joke "Why do bears have hairy coats? Because they look silly in sweaters!" operates on multiple levels to deliver humor effectively.\n\n1. **Play on Words**: The setup creates an expectation for a sensible, biological explanation for why bears are covered in fur. However, the punchline surprises the audience with a whimsical reason, attributing fashion sense and human concerns about appearance to bears, which is inherently absurd.\n\n2. **Imagery**: The mental image of a bear attempting to wear a sweater is amusing due to the stark contrast between the majestic, wild nature of bears and the mundane concept of wearing clothing. This juxtaposition amplifies the humor as it defies the audience\'s expectations of animal behavior.\n\n3. **Subversion of Expectations**: The joke cleverly subverts the listener\'s anticipation of a straightforward biological answer by offering a silly and illogical response instead. This twist creates an element of surprise, which is a fundam